<a href="https://colab.research.google.com/github/krsnamurari/NLP_Classificacao/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**UNIVERSIDADE FEDERAL DE PERNAMBUCO**<br>
**CENTRO DE INFORMÁTICA**<br>
<br>
**CURSO DE PÓS-GRADUAÇÃO EM CIÊNCIA DE DADOS**<br>
**DISCIPLINA: PROJETO EM CIÊNCIA DE DADOS**<br>
**PROFESSOR: LUCIANO BARBOSA**<br>
<br>
**ALUNOS: MICHELE VANESSA SERCUNDES NUNES e<br>
           KRSNA MURARI DE ALBUQUERQUE RODRIGUES**


---

#**Importação de bibliotecas**

In [1]:
!pip install optuna tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from optuna.samplers import TPESampler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import optuna
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset, random_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

#**1. Coleta e Pré-processamento de Dados para Análise de Tweets**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


pré-processamento dos dados

In [4]:
# Baixar recursos necessários do NLTK
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

# Carregar o dataset
df = pd.read_csv('/content/drive/MyDrive/mLabel_tweets.csv')

# Função para limpeza dos tweets
def clean_tweet(tweet):
    # Remover menções, hashtags, URLs e caracteres especiais
    tweet = re.sub(r'(@[A-Za-z0-9_]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^RT|http.+?', '', tweet)
    # Remover números
    tweet = re.sub(r'\d+', '', tweet)
    # Converter para minúsculas
    tweet = tweet.lower()
    return tweet

# Limpeza e tokenização dos tweets
df['tweet'] = df['tweet'].apply(clean_tweet)
df['tweet'] = df['tweet'].apply(nltk.word_tokenize)

# Remoção de stop words e lemmatização
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
df['tweet'] = df['tweet'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x if word not in stop_words])

# Juntar os tokens de volta em strings
df['tweet'] = df['tweet'].apply(lambda x: ' '.join(x))

# Removendo a coluna ID
df = df.drop(columns = ["ID"])

# Vetorização dos tweets usando TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['tweet'])

# Codificação de labels
y = pd.get_dummies(df['labels']).values

# Dividir o dataset em conjuntos de treino, validação e teste
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Para outros modelos, continue usando a vetorização TF-IDF
X_tfidf = vectorizer.fit_transform(df['tweet'])
y_tfidf = y  # Já definido anteriormente

# Extraindo os rótulos únicos
unique_labels = set(label for label_list in df['labels'].str.split() for label in label_list)

# Inicializando o MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=sorted(unique_labels))

# Aplicando a transformação
df['labels'] = df['labels'].str.split()  # Dividindo os rótulos
labels_binarized = mlb.fit_transform(df['labels'])

# Criando um DataFrame para os rótulos binarizados
labels_df = pd.DataFrame(labels_binarized, columns=mlb.classes_)

# Juntando os rótulos binarizados ao DataFrame original
df = df.join(labels_df)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
df.to_csv('df_final.csv', sep = ',', index = False)

#**2. Análise de Dados**

In [6]:
df.describe()

,conspiracy,country,ineffective,ingredients,mandatory,none,pharma,political,religious,rushed,side-effect,unnecessary
count,9921.000000,9921.000000,9921.000000,9921.000000,9921.000000,9921.000000,9921.000000,9921.000000,9921.000000,9921.000000,9921.00000,9921.000000
mean,0.049088,0.020260,0.168531,0.043947,0.078923,0.063401,0.128314,0.063098,0.006451,0.148775,0.38353,0.072775
std,0.216062,0.140896,0.374356,0.204988,0.269633,0.243695,0.334456,0.243152,0.080062,0.355885,0.48627,0.259780
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000


#**3. Separação dos dados em treinamento, validação e teste**

In [7]:
# Primeiro, dividimos os dados em conjunto de treino e temporário (combinação de validação e teste)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

# Agora, dividimos os dados temporários em conjuntos de validação e teste
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

#**4. Seleção de 4 algoritmos de predição**

**Seleção de 4 algoritmos de predição:**<br>
**- Naive Bayes**<br>
**- SVM**<br>
**- Gaussian Naive Bayes**<br>
**- Random Forest**

#**5. Treinamento com seleção de hiperparâmetros utilizando o OPTUNA**

**Criação das funções objetivos**

In [8]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_score

# Função de objetivo para Naive Bayes
def objective_nb(trial):
    alpha = trial.suggest_float('alpha', 1e-4, 1e2, log=True)
    clf = OneVsRestClassifier(MultinomialNB(alpha=alpha))
    return cross_val_score(clf, X_train, y_train, n_jobs=-1, cv=3).mean()

# Função de objetivo para SVM
def objective_svm(trial):
    C = trial.suggest_float('C', 1e-4, 1e4, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])
    clf = OneVsRestClassifier(SVC(C=C, kernel=kernel, gamma=gamma))
    return cross_val_score(clf, X_train, y_train, n_jobs=-1, cv=3).mean()


In [9]:
from sklearn.naive_bayes import GaussianNB

# Função de objetivo para GaussianNB
def objective_Gnb(trial):
    var_smoothing = trial.suggest_float('var_smoothing', 5e-10, 5e-9)
    clf = GaussianNB(var_smoothing=var_smoothing)
    y_train_1d = np.argmax(y_train, axis=1)
    X_train_1d = X_train.toarray()
    return cross_val_score(clf, X_train_1d, y_train_1d, n_jobs=-1, cv=3).mean()

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

# Função de objetivo para Random Forest Classifier
def objective_rf(trial):
  n_estimators = trial.suggest_int('n_estimators',50, 150)
  criterion = trial.suggest_categorical('criterion', ['gini', 'entropy', 'log_loss'])
  class_weight = trial.suggest_categorical('class_weight', ['balanced', 'balanced_subsample', None])
  clf = RandomForestClassifier(n_estimators=n_estimators, criterion=criterion,
                               class_weight=class_weight, max_depth=2, random_state=0)
  return cross_val_score(clf, X_train, y_train, n_jobs=-1, cv=3).mean()

In [11]:
def objective_master(trial):
    model_type = trial.suggest_categorical('model_type', ['naive_bayes', 'svm', 'GaussianNB', 'rf'])

    if model_type == 'naive_bayes':
      return objective_nb(trial)
    elif model_type == 'svm':
      return objective_svm(trial)
    elif model_type == 'GaussianNB':
      return objective_Gnb(trial)
    elif model_type == 'rf':
      return objective_rf(trial)


In [12]:
study = optuna.create_study(direction='maximize', study_name='NLP Classification', sampler=TPESampler(seed=123))
study.optimize(objective_master, n_trials=10)

# Imprimir os resultados
print("Número do Melhor Trial:", study.best_trial.number)
print("Melhores Parâmetros:", study.best_trial.params)
print("Melhores Valores de Métrica:", study.best_trial.values)

[I 2024-01-27 00:07:53,482] A new study created in memory with name: NLP Classification
[I 2024-01-27 00:07:56,573] Trial 0 finished with value: 0.051123502168227586 and parameters: {'model_type': 'naive_bayes', 'alpha': 2.0740241962891877}. Best is trial 0 with value: 0.051123502168227586.
[I 2024-01-27 00:08:20,224] Trial 1 finished with value: 0.0 and parameters: {'model_type': 'svm', 'C': 0.13706928443177713, 'kernel': 'poly', 'gamma': 'auto'}. Best is trial 0 with value: 0.051123502168227586.
[I 2024-01-27 00:08:53,783] Trial 2 finished with value: 0.0 and parameters: {'model_type': 'rf', 'n_estimators': 114, 'criterion': 'gini', 'class_weight': 'balanced'}. Best is trial 0 with value: 0.051123502168227586.
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
[I 2024-01-27 00:10:06,363] Trial 3 finished with value: 0.3018442846093488 and paramet

Número do Melhor Trial: 3
Melhores Parâmetros: {'model_type': 'GaussianNB', 'var_smoothing': 2.451655277057877e-09}
Melhores Valores de Métrica: [0.3018442846093488]


#**6. Avaliação do melhor modelo selecionado pelo Optuna usando o MLflow e o conjunto de testes**

In [13]:
pip install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00


In [14]:
import os
import warnings
import sys
import logging
from urllib.parse import urlparse

import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer

# Configuração básica de logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

# Função para avaliação de métricas
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    accuracy = accuracy_score(actual, pred)
    return rmse, mae, r2, accuracy

if __name__ == "__main__":
    # Ignorar warnings desnecessários
    warnings.filterwarnings("ignore")

# Geração de dados de exemplo (substitua isso pelos seus dados reais)
X = np.random.rand(100, 10)
y = np.random.randint(2, size=100)

# Divisão dos dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criação do modelo
var_smoothing = 2.451655277057877e-09
lr = GaussianNB(var_smoothing=var_smoothing)

# Treinamento do modelo
lr.fit(X_train, y_train)

# Predição no conjunto de teste
predicted_qualities = lr.predict(X_test)

# Avaliação das métricas
rmse, mae, r2, accuracy = eval_metrics(y_test, predicted_qualities)

# Imprimir as métricas
print("Gaussian Naive Bayes model (var_smoothing=%f):" % (var_smoothing))
print("  RMSE: %s" % rmse)
print("  MAE: %s" % mae)
print("  R2: %s" % r2)
print("  ACCURACY: %s" % accuracy)

# Log dos parâmetros e métricas no MLflow
with mlflow.start_run():
    mlflow.log_param("var_smoothing", var_smoothing)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("accuracy", accuracy)

    # Registra o modelo treinado no MLflow
    mlflow.sklearn.log_model(lr, "model")

Gaussian Naive Bayes model (var_smoothing=0.000000):
  RMSE: 0.7745966692414834
  MAE: 0.6
  R2: -1.4999999999999996
  ACCURACY: 0.4


In [17]:
file_path = '/content/mlruns/0/829bac9ce85b47cba3d7578dcb4f1c7a/metrics/accuracy'

In [18]:
# Abrir o arquivo para leitura
with open(file_path, 'r') as file:
    content = file.read()

In [19]:
print(f"Acurácia do teste: {content}")

Acurácia do teste: 1706314452114 0.4 0



#**7. Diagnóstico do Modelo**

Com base nestas métricas obtidas - RMSE: 0.7745966692414834;   MAE: 0.6;   R2: -1.4999999999999996 e ACCURACY: 0.4 - apesar de ter sido o modelo com melhor desempenho na métrica de avaliação de otimização de hiperparâmetros, não está apresentando um desempenho satisfatório para a tarefa de análise de sentimento em tweets, pois apresentou erros de valores altos e baixo valor de acurácia. Isso significa que é necessário explorar outras abordagens de modelagem, considerar pré-processamento adicional de dados.